<a href="https://colab.research.google.com/github/zhenxiay/GoogleColab/blob/main/PySpark_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=90e3d1ee8235beb18c21e685f671a1d9d08e6f96d743b605722aac633cdd1bb0
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pyspark.pandas as ps

/usr/local/lib/python3.10/dist-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [ ]:
spark = SparkSession \
        .builder \
        .appName("firstSpark") \
        .getOrCreate()

In [ ]:
path='https://raw.githubusercontent.com/PacktPublishing/Python-for-Finance-Cookbook/master/Datasets/credit_card_default.csv'

In [ ]:
df = spark.createDataFrame(pd.read_csv(path))

In [ ]:
df.printSchema()
df.count()

root
 |-- Unnamed: 0: long (nullable = true)
 |-- limit_bal: long (nullable = true)
 |-- sex: string (nullable = true)
 |-- education: string (nullable = true)
 |-- marriage: string (nullable = true)
 |-- age: double (nullable = true)
 |-- payment_status_sep: string (nullable = true)
 |-- payment_status_aug: string (nullable = true)
 |-- payment_status_jul: string (nullable = true)
 |-- payment_status_jun: string (nullable = true)
 |-- payment_status_may: string (nullable = true)
 |-- payment_status_apr: string (nullable = true)
 |-- bill_statement_sep: long (nullable = true)
 |-- bill_statement_aug: long (nullable = true)
 |-- bill_statement_jul: long (nullable = true)
 |-- bill_statement_jun: long (nullable = true)
 |-- bill_statement_may: long (nullable = true)
 |-- bill_statement_apr: long (nullable = true)
 |-- previous_payment_sep: long (nullable = true)
 |-- previous_payment_aug: long (nullable = true)
 |-- previous_payment_jul: long (nullable = true)
 |-- previous_payment_jun: 

30000

In [ ]:
df.show(5,0)

+----------+---------+------+----------+--------+----+------------------------+------------------------+------------------+------------------+------------------+------------------------+------------------+------------------+------------------+------------------+------------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------------+
|Unnamed: 0|limit_bal|sex   |education |marriage|age |payment_status_sep      |payment_status_aug      |payment_status_jul|payment_status_jun|payment_status_may|payment_status_apr      |bill_statement_sep|bill_statement_aug|bill_statement_jul|bill_statement_jun|bill_statement_may|bill_statement_apr|previous_payment_sep|previous_payment_aug|previous_payment_jul|previous_payment_jun|previous_payment_may|previous_payment_apr|default_payment_next_month|
+----------+---------+------+----------+--------+----+------------------------+-------------

In [ ]:
df.write.format("parquet").mode("overwrite").saveAsTable("credit_risk")

In [ ]:
df.dropna().groupBy('Sex').agg(count('age').alias('count'),avg('default_payment_next_month').alias('avg_default')).show()

+------+-----+-------------------+
|   Sex|count|        avg_default|
+------+-----+-------------------+
|Female|17933|0.20715998438632688|
|   NaN|  150|0.24666666666666667|
|  Male|11767|0.24211778703152886|
+------+-----+-------------------+



In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder

In [ ]:
stringIndexer = StringIndexer(inputCol="payment_status_sep",
                              outputCol="paymentStatusIndexSep")
model = StringIndexer.fit(df)
indexed_df = model.transform(df)

TypeError: Estimator.fit() missing 1 required positional argument: 'dataset'

In [ ]:
encoder = OneHotEncoder(inputCol="paymentStatusIndexSep", outputCol="PaymentStatusIndexSepVec")
encoded_df = encoder.transform(indexed_df)

AttributeError: 'OneHotEncoder' object has no attribute 'transform'

In [ ]:
indexed_df.show()

+----------+---------+------+---------------+--------+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+------------------+------------------+------------------+------------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------------+---------------------+
|Unnamed: 0|limit_bal|   sex|      education|marriage| age|  payment_status_sep|  payment_status_aug|  payment_status_jul|  payment_status_jun|  payment_status_may|  payment_status_apr|bill_statement_sep|bill_statement_aug|bill_statement_jul|bill_statement_jun|bill_statement_may|bill_statement_apr|previous_payment_sep|previous_payment_aug|previous_payment_jul|previous_payment_jun|previous_payment_may|previous_payment_apr|default_payment_next_month|paymentStatusIndexSep|
+----------+---------+------+---------------+-----

In [ ]:
df.dropna().show()

+----------+---------+------+---------------+--------+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+------------------+------------------+------------------+------------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------------+
|Unnamed: 0|limit_bal|   sex|      education|marriage| age|  payment_status_sep|  payment_status_aug|  payment_status_jul|  payment_status_jun|  payment_status_may|  payment_status_apr|bill_statement_sep|bill_statement_aug|bill_statement_jul|bill_statement_jun|bill_statement_may|bill_statement_apr|previous_payment_sep|previous_payment_aug|previous_payment_jul|previous_payment_jun|previous_payment_may|previous_payment_apr|default_payment_next_month|
+----------+---------+------+---------------+--------+----+--------------------+--------------

In [ ]:
df_agg = df.dropna().groupBy('payment_status_sep').agg(count('age').alias('count'),avg('default_payment_next_month').alias('avg_default'))

In [ ]:
df_agg.orderBy(df_agg.payment_status_sep.desc()).show()

+--------------------+-----+-------------------+
|  payment_status_sep|count|        avg_default|
+--------------------+-----+-------------------+
|             Unknown|17413|0.12881180727043015|
|Payment delayed 8...|   18| 0.6111111111111112|
|Payment delayed 7...|    9| 0.7777777777777778|
|Payment delayed 6...|   11| 0.5454545454545454|
|Payment delayed 5...|   26|                0.5|
|Payment delayed 4...|   76| 0.6842105263157895|
|Payment delayed 3...|  319| 0.7617554858934169|
|Payment delayed 2...| 2649| 0.6919592298980748|
|Payment delayed 1...| 3666| 0.3393344244408074|
|          Payed duly| 5663|0.16757902171993644|
+--------------------+-----+-------------------+



In [ ]:
import plotly.express as px

df_agg_pd = df_agg.orderBy(df_agg.payment_status_sep.desc()).toPandas()

fig = px.bar(df_agg_pd, x='payment_status_sep', y='avg_default', text='avg_default')
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.show()

In [ ]:
df.dropna().cube(df.sex,df.education,df.marriage).avg("age").show()

+------+---------------+--------+------------------+
|   sex|      education|marriage|          avg(age)|
+------+---------------+--------+------------------+
|Female|     University|  Others|              40.5|
|  NULL|           NULL| Married| 40.02337623908862|
|Female|            NaN|  Single|  30.3953488372093|
|  NULL|           NULL|    NULL| 35.49011725293132|
|  Male|           NULL|  Single| 32.16439205955335|
|  Male|         Others|    NULL|36.970238095238095|
|   NaN|           NULL|  Single|30.948051948051948|
|  NULL|     University|  Others|40.544910179640716|
|  NULL|    High school|  Others| 43.97931034482759|
|Female|            NaN|    NULL|33.833333333333336|
|  NULL|         Others|    NULL| 36.13763440860215|
|  NULL|           NULL|  Others| 42.01340482573727|
|   NaN|           NULL|  Others|43.666666666666664|
|Female|     University|    NULL| 34.20874868066143|
|  NULL|    High school|     NaN| 42.13333333333333|
|  Male|           NULL|    NULL| 36.520098580

In [ ]:
from pyspark.sql.functions import min, max

oldest = df.select(max("age")).collect()[0][0]
youngest = df.select(min("age")).collect()[0][0]

print("Oldest pickup date: ", oldest)
print("Latest pickup date: ", youngest)

Oldest pickup date:  nan
Latest pickup date:  21.0


In [ ]:
# min max values of target feature "fare_amount"

df.select(min('bill_statement_sep').alias('min'), max('bill_statement_sep').alias('max')).show()

+-------+------+
|    min|   max|
+-------+------+
|-165580|964511|
+-------+------+



In [ ]:
df_cube = ps.DataFrame(df.dropna().cube(df.sex,df.education,df.marriage).avg("age"))

In [ ]:
df_cube

,sex,education,marriage,avg(age)
0,Female,University,Others,40.500000
1,None,None,Married,40.023376
2,Female,NaN,Single,30.395349
3,None,None,None,35.490117
4,Male,None,Single,32.164392
5,Male,Others,None,36.970238
6,NaN,None,Single,30.948052
7,None,University,Others,40.544910
8,None,High school,Others,43.979310
9,Female,NaN,None,33.833333


In [ ]:
df.createOrReplaceTempView("table1")

query = "SELECT education, round(avg(limit_bal),2) as avg_credit_limit from table1 where education != 'NaN' group by education"

df_sql = spark.sql(query)

In [ ]:
df_sql.show()

+---------------+----------------+
|      education|avg_credit_limit|
+---------------+----------------+
|Graduate school|       213016.04|
|         Others|        181383.3|
|    High school|       126554.99|
|     University|       147082.81|
+---------------+----------------+



In [ ]:
ps.DataFrame(df_sql).plot.bar(x='education',y='avg_credit_limit')

In [ ]:
spark.sql("SELECT * FROM table1 WHERE education = 'University' and sex = 'Female'").show()